<a href="https://colab.research.google.com/github/virf96/Chat-Bot/blob/master/CodificacionVariables_Codificacion_One_hot_categoria_frecuente.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Codificación One-Hot de categorías frecuentes ( One-Hot encoding OHE ) 

Aprendimos en la sección 3, que la alta cardinalidad y las etiquetas poco comunes pueden generar que algunas categorías solo aparezcan en el set de entrenamiento y por lo tanto causar sobreajustes. También aprendimos que si las categorías solo aparecen en el set de prueba, nuestros modelos no sabrán como evaluar esas observaciones

Adicionalmente aprendimos que en la codificación one-hot, si las variables categóricas tienen múltiples etiquetas, el re-codificarlas con variables dummy puede expandir el espacio de los datos considerablemente.

** Para evitar estas complicaciones, podemos crear variables dummy solo para las categorías más frecuentes**

Este proceso también se conoce como codificación one-hot de las categorías más frecuentes.

De hecho, en la solución ganadora de la competencia KDD 2009 : ["Winning the KDD Cup Orange Challenge with Ensemble Selection"](http://www.mtome.com/Publications/CiML/CiML-v3-book.pdf), los autores limitaron la codificación one-hot a las 10 categorías más frecuentes de la variable. Esto significa que solamente se genera una variable binaria por cada una de las 10 categorías más frecuentes.

OHE de categorías más frecuentes es equivalente a agrupar las categorías restantes o menos frecuentes bajo una nueva categoría. 

En un notebook más adelante de esta sección, veremos como mejor agrupar las etiquetas poco frecuentes en una nueva categoría.


### Ventajas de OHE de las categorías más frecuentes

- Fácil de implementar
- No requiere horas en exploración de las variables
- No expande masivamente el espacio de los datos (número de variables)
- Es adecuado para modelos lineales

### Limitaciones

- No añade ninguna información que puede hacer que la  variable tenga mas poder predictivo
- No retiene información sobre las etiquetas ignoradas o descartadas.

Frecuentemente, las variables categóricas tienen unas categorías más dominantes mientras que las categorías restantes añaden poca información. Por lo tanto, OHE de las categorías más frecuentes es una técnica sencilla y a la vez muy útil

### Nota

El número de variables más frecuentes se fija arbitrariamente. En la competencia KDD los autores seleccionaron 10, pero bien pudo haber sido 15 o 5. Este número puede ser escogido arbitrariamente o derivado de la exploración de los datos.


## En este demo:

Vamos a realizar codificación one-hot con:
- pandas y NumPy
- Feature-Engine

y veremos las ventajas y limitaciones de cada una de estas implementaciones usando los datos House Prices dataset. 


In [1]:
pip install feature_engine

     |████████████████████████████████| 9.5MB 6.9MB/s 
  Found existing installation: statsmodels 0.10.2
    Uninstalling statsmodels-0.10.2:
      Successfully uninstalled statsmodels-0.10.2


In [2]:
import numpy as np
import pandas as pd

# para separar el conjunto de datos
from sklearn.model_selection import train_test_split

# one hot encoding con feature-engine
from feature_engine.categorical_encoders import OneHotCategoricalEncoder

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# cargar dataset

data = pd.read_csv(
    '/content/drive/My Drive/datasets/houseprice.csv',
    usecols=['Neighborhood', 'Exterior1st', 'Exterior2nd', 'SalePrice'])

data.head()

,Neighborhood,Exterior1st,Exterior2nd,SalePrice
0,CollgCr,VinylSd,VinylSd,208500
1,Veenker,MetalSd,MetalSd,181500
2,CollgCr,VinylSd,VinylSd,223500
3,Crawfor,Wd Sdng,Wd Shng,140000
4,NoRidge,VinylSd,VinylSd,250000


In [5]:
# miremos cuantas etiquetas tiene cada variable

for col in data.columns:
    print(col, ': ', len(data[col].unique()), ' etiquetas')

Neighborhood :  25  etiquetas
Exterior1st :  15  etiquetas
Exterior2nd :  16  etiquetas
SalePrice :  663  etiquetas


In [6]:
# exploremos cada una de las categorías de las variables
data['Neighborhood'].unique()

array(['CollgCr', 'Veenker', 'Crawfor', 'NoRidge', 'Mitchel', 'Somerst',
       'NWAmes', 'OldTown', 'BrkSide', 'Sawyer', 'NridgHt', 'NAmes',
       'SawyerW', 'IDOTRR', 'MeadowV', 'Edwards', 'Timber', 'Gilbert',
       'StoneBr', 'ClearCr', 'NPkVill', 'Blmngtn', 'BrDale', 'SWISU',
       'Blueste'], dtype=object)

In [7]:
data['Exterior1st'].unique()

array(['VinylSd', 'MetalSd', 'Wd Sdng', 'HdBoard', 'BrkFace', 'WdShing',
       'CemntBd', 'Plywood', 'AsbShng', 'Stucco', 'BrkComm', 'AsphShn',
       'Stone', 'ImStucc', 'CBlock'], dtype=object)

In [8]:
data['Exterior2nd'].unique()

array(['VinylSd', 'MetalSd', 'Wd Shng', 'HdBoard', 'Plywood', 'Wd Sdng',
       'CmentBd', 'BrkFace', 'Stucco', 'AsbShng', 'Brk Cmn', 'ImStucc',
       'AsphShn', 'Stone', 'Other', 'CBlock'], dtype=object)

### Nota importante sobre codificación

Es importante seleccionar las categorías más frecuentes basados en el set de entrenamiento; luego usaremos estas categorías para codificar las variables en el set de prueba.


In [9]:
# separemos en set de prueba y entrenamiento

X_train, X_test, y_train, y_test = train_test_split(
    data[['Neighborhood', 'Exterior1st', 'Exterior2nd']],  # predictores
    data['SalePrice'],  # target
    test_size=0.3,  # porcentaje observaciones prueba
    random_state=0)  # seed/semilla asegurar reproducibilidad

X_train.shape, X_test.shape

((1022, 3), (438, 3))

In [10]:
# primero examinemos como OHE expande el espacio de los datos 
# (incrementa el número de variables)

pd.get_dummies(X_train, drop_first=True).shape

(1022, 53)

De las 3 variables categóricas que teníamos originalmente, terminamos con 53 variables luego de la transformación.

53 variables no es un número muy grande de variables, y todavía es relativamente sencillo manejarlas. Sin embargo, en datos de conjuntos reales, las variables categórias pueden ser altamente cardinales y con OHE podemos terminar con datos de miles de variables.


## OHE para categorías más frecuentes  con pandas y NumPy


### Ventajas

- rápido
- devuelve un pandas dataframe
- devuelve los nombres de las variables para las variables dummy

### Limitaciones:

- no preserva la información del set de entrenamiento para propagarlo al set de prueba

In [11]:
# busquemos las 10 categorías más frecuentes para la variable 'Neighborhood'

X_train['Neighborhood'].value_counts().sort_values(ascending=False).head(10)

NAmes      151
CollgCr    105
OldTown     73
Edwards     71
Sawyer      61
Somerst     56
Gilbert     55
NWAmes      51
NridgHt     51
SawyerW     45
Name: Neighborhood, dtype: int64

In [12]:
# creemos una lista con las categorías más frecuentes 

# selecciones las primeras 10

top_10 = [
    x for x in X_train['Neighborhood'].value_counts().sort_values(
        ascending=False).head(10).index
]

top_10

['NAmes',
 'CollgCr',
 'OldTown',
 'Edwards',
 'Sawyer',
 'Somerst',
 'Gilbert',
 'NWAmes',
 'NridgHt',
 'SawyerW']

In [14]:
# y ahora procedamos a crear las 10 variables binarias

for label in top_10:
    X_train['Neighborhood' + '_' + label] = np.where(
        X_train['Neighborhood'] == label, 1, 0)
    
    X_test['Neighborhood' + '_' + label] = np.where(
        X_test['Neighborhood'] == label, 1, 0)

# visualicemos los resultados
X_train[['Neighborhood'] + ['Neighborhood'+'_'+c for c in top_10]].head(10)

,Neighborhood,Neighborhood_NAmes,Neighborhood_CollgCr,Neighborhood_OldTown,Neighborhood_Edwards,Neighborhood_Sawyer,Neighborhood_Somerst,Neighborhood_Gilbert,Neighborhood_NWAmes,Neighborhood_NridgHt,Neighborhood_SawyerW
64,CollgCr,0,1,0,0,0,0,0,0,0,0
682,ClearCr,0,0,0,0,0,0,0,0,0,0
960,BrkSide,0,0,0,0,0,0,0,0,0,0
1384,Edwards,0,0,0,1,0,0,0,0,0,0
1100,SWISU,0,0,0,0,0,0,0,0,0,0
416,Sawyer,0,0,0,0,1,0,0,0,0,0
1034,Crawfor,0,0,0,0,0,0,0,0,0,0
853,NAmes,1,0,0,0,0,0,0,0,0,0
472,Edwards,0,0,0,1,0,0,0,0,0,0
1011,Edwards,0,0,0,1,0,0,0,0,0,0


In [15]:
# los comandos anteriores se pueden simplificar creando 2 funciones

def calculate_top_categories(df, variable, how_many=10):
    return [
        x for x in df[variable].value_counts().sort_values(
            ascending=False).head(how_many).index
    ]


def one_hot_encode(train, test, variable, top_x_labels):

    for label in top_x_labels:
        train[variable + '_' + label] = np.where(
            train['Neighborhood'] == label, 1, 0)
        
        test[variable + '_' + label] = np.where(
            test['Neighborhood'] == label,1, 0)

In [16]:
calculate_top_categories(X_train,'Neighborhood',2)

['NAmes', 'CollgCr']

In [17]:
# y ahora repitamos el proceso para el resto de las variables categóricas

for variable in ['Exterior1st', 'Exterior2nd']:
    
    top_categories = calculate_top_categories(X_train, variable, how_many=10)
    
    one_hot_encode(X_train, X_test, variable, top_categories)

In [18]:
# veamos el resultado

X_train.head()

,Neighborhood,Exterior1st,Exterior2nd,Neighborhood_NAmes,Neighborhood_CollgCr,Neighborhood_OldTown,Neighborhood_Edwards,Neighborhood_Sawyer,Neighborhood_Somerst,Neighborhood_Gilbert,Neighborhood_NWAmes,Neighborhood_NridgHt,Neighborhood_SawyerW,Exterior1st_VinylSd,Exterior1st_HdBoard,Exterior1st_Wd Sdng,Exterior1st_MetalSd,Exterior1st_Plywood,Exterior1st_CemntBd,Exterior1st_BrkFace,Exterior1st_WdShing,Exterior1st_Stucco,Exterior1st_AsbShng,Exterior2nd_VinylSd,Exterior2nd_Wd Sdng,Exterior2nd_HdBoard,Exterior2nd_MetalSd,Exterior2nd_Plywood,Exterior2nd_CmentBd,Exterior2nd_Wd Shng,Exterior2nd_BrkFace,Exterior2nd_AsbShng,Exterior2nd_Stucco
64,CollgCr,VinylSd,VinylSd,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
682,ClearCr,Wd Sdng,Wd Sdng,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
960,BrkSide,Wd Sdng,Plywood,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1384,Edwards,WdShing,Wd Shng,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1100,SWISU,Wd Sdng,Wd Sdng,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [21]:
len(top_categories)

10

In [19]:
X_train.shape

(1022, 33)

Fíjate como ahora tenemos 30 variables dummy adicionales en lugar de las 53 que hubiésemos tenido si hubiésemos creado una variable dummy por cada una de las categorías

## OHE para categorías más frecuentes con Feature-Engine


### Ventajas

- rápido
- crea el mismo número de variables en el set de entrenamiento y prueba

### Limitaciones:

- Ninguna que sepamos! :)

In [22]:
# separemos los datos en sets de prueba y entrenamiento

X_train, X_test, y_train, y_test = train_test_split(
    data[['Neighborhood', 'Exterior1st', 'Exterior2nd']],  # variables
    data['SalePrice'],  # target
    test_size=0.3,  # porcentaje de observaciones prueba
    random_state=0)  # seed/semilla para asegurar reproducibilidad

X_train.shape, X_test.shape

((1022, 3), (438, 3))

In [23]:
ohe_enc = OneHotCategoricalEncoder(
    top_categories=10,  # puedes cambiar este valor para seleccionar mas o menos variables
    # puedes seleccionar cuales variables codificar
    variables=['Neighborhood', 'Exterior1st', 'Exterior2nd'],
    drop_last=False) #Para que no nos quite columnas

ohe_enc.fit(X_train)

OneHotCategoricalEncoder(drop_last=False, top_categories=10,
                         variables=['Neighborhood', 'Exterior1st',
                                    'Exterior2nd'])

In [24]:
# en el dict del encoder podemos ver cada una de las categorías más frecuentes
# seleccionadas para cada una de las variables

ohe_enc.encoder_dict_

{'Exterior1st': ['VinylSd',
  'HdBoard',
  'Wd Sdng',
  'MetalSd',
  'Plywood',
  'CemntBd',
  'BrkFace',
  'WdShing',
  'Stucco',
  'AsbShng'],
 'Exterior2nd': ['VinylSd',
  'Wd Sdng',
  'HdBoard',
  'MetalSd',
  'Plywood',
  'CmentBd',
  'Wd Shng',
  'BrkFace',
  'AsbShng',
  'Stucco'],
 'Neighborhood': ['NAmes',
  'CollgCr',
  'OldTown',
  'Edwards',
  'Sawyer',
  'Somerst',
  'Gilbert',
  'NWAmes',
  'NridgHt',
  'SawyerW']}

In [25]:
# esta es la lista de las variables que el encoder transformará

ohe_enc.variables

['Neighborhood', 'Exterior1st', 'Exterior2nd']

In [26]:
X_train = ohe_enc.transform(X_train)
X_test = ohe_enc.transform(X_test)

# exploremos el resultado
X_train.head()

,Neighborhood_NAmes,Neighborhood_CollgCr,Neighborhood_OldTown,Neighborhood_Edwards,Neighborhood_Sawyer,Neighborhood_Somerst,Neighborhood_Gilbert,Neighborhood_NWAmes,Neighborhood_NridgHt,Neighborhood_SawyerW,Exterior1st_VinylSd,Exterior1st_HdBoard,Exterior1st_Wd Sdng,Exterior1st_MetalSd,Exterior1st_Plywood,Exterior1st_CemntBd,Exterior1st_BrkFace,Exterior1st_WdShing,Exterior1st_Stucco,Exterior1st_AsbShng,Exterior2nd_VinylSd,Exterior2nd_Wd Sdng,Exterior2nd_HdBoard,Exterior2nd_MetalSd,Exterior2nd_Plywood,Exterior2nd_CmentBd,Exterior2nd_Wd Shng,Exterior2nd_BrkFace,Exterior2nd_AsbShng,Exterior2nd_Stucco
64,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
682,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
960,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
1384,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0
1100,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0


In [27]:
X_test.head()

,Neighborhood_NAmes,Neighborhood_CollgCr,Neighborhood_OldTown,Neighborhood_Edwards,Neighborhood_Sawyer,Neighborhood_Somerst,Neighborhood_Gilbert,Neighborhood_NWAmes,Neighborhood_NridgHt,Neighborhood_SawyerW,Exterior1st_VinylSd,Exterior1st_HdBoard,Exterior1st_Wd Sdng,Exterior1st_MetalSd,Exterior1st_Plywood,Exterior1st_CemntBd,Exterior1st_BrkFace,Exterior1st_WdShing,Exterior1st_Stucco,Exterior1st_AsbShng,Exterior2nd_VinylSd,Exterior2nd_Wd Sdng,Exterior2nd_HdBoard,Exterior2nd_MetalSd,Exterior2nd_Plywood,Exterior2nd_CmentBd,Exterior2nd_Wd Shng,Exterior2nd_BrkFace,Exterior2nd_AsbShng,Exterior2nd_Stucco
529,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
491,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
459,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
279,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
655,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


**Nota**

Si el argumento 'variables' se fija en 'None' (ninguno)  el codificador automáticamente indentificará  **todas las variables categóricas**. Maravilloso verdad?

El codificador no codificará las variables numéricas. Entonces si algunas de tus variables categóricas son de hecho numéricas, necesita hacer el 're-cast' o cambio a tipo 'object' antes de usar el codificador.
